In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import pytz
import numpy as np

from models.gan_models import *
import yfinance as yf
import MetaTrader5 as mt5

import os

In [4]:
#api_key = os.environ['COMET_API_KEY']
api_key = 'vpNJF6XOWcHS6HqH9ZFEjwRcD'

In [5]:
experiment = Experiment(
  api_key=api_key,
  project_name="tail-price",
  workspace="artaasd95"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information
COMET INFO: Experiment is live on comet.com https://www.comet.com/artaasd95/tail-price/0e3b3779ac364200aa2f89e9bfba43c6



In [21]:
mt5.initialize()

print('loading current tf data')
utc_from = datetime(2021, 1, 1, tzinfo=pytz.timezone("Asia/Nicosia"))
utc_to = datetime.now(pytz.timezone("Asia/Nicosia"))

data = mt5.copy_rates_range('XAUUSD', mt5.TIMEFRAME_H4, utc_from, utc_to)
data = pd.DataFrame(data)
time_data = data.time
data.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)

loading current tf data


In [23]:
data.to_csv('xau_2021_H4.csv')

In [5]:
data = pd.read_csv('xau_2021_H4.csv')

In [6]:
data

,Unnamed: 0,time,open,high,low,close
0,0,1609718400,1904.48,1918.60,1900.62,1916.57
1,1,1609732800,1916.65,1925.15,1915.44,1921.56
2,2,1609747200,1921.52,1935.09,1921.34,1932.03
3,3,1609761600,1931.95,1942.06,1927.79,1939.75
4,4,1609776000,1939.78,1944.33,1929.23,1936.92
...,...,...,...,...,...,...
5582,5582,1723780800,2458.38,2459.97,2450.72,2452.68
5583,5583,1723795200,2452.68,2464.50,2451.11,2462.32
5584,5584,1723809600,2462.32,2492.34,2461.17,2491.49
5585,5585,1723824000,2491.46,2500.08,2477.46,2495.82


In [7]:
input_size = 1
hidden_size = 1024
seq_length = 50
num_layers = 12
batch_size = 32
num_epochs = 100
learning_rate = 0.3

# Initialize models
main_gen = MainGenerator(input_size, hidden_size, num_layers, 0.5, 1)
noise_gen = NoiseGenerator(input_size, 2048, 1, 0.4)
discriminator = Discriminator(input_size, hidden_size, 15, 0.4)

In [8]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [9]:
main_gen.to(device)
noise_gen.to(device)
discriminator.to(device)

Discriminator(
  (lstm): LSTM(1, 1024, num_layers=15, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (sigmoid): Sigmoid()
)

In [10]:
optimizer_G = torch.optim.AdamW(list(main_gen.parameters()) + list(noise_gen.parameters()), lr=learning_rate)
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=learning_rate)

optim_g_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, 0.1)
optim_d_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, 0.1)

In [11]:
cauchy_dist = torch.distributions.cauchy.Cauchy(loc=0, scale=0.5)

In [12]:
#adversarial_loss = F.kl_div
adversarial_loss = F.mse_loss

In [46]:
train_data = data[:int(len(data)*0.7)]
train_data = train_data.reset_index(drop=True)
test_data = data[int(len(data)*0.7):]
test_data = test_data.reset_index(drop=True)

In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    batch_start = 0
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(train_data)/batch_size)-1):
        price = torch.tensor(train_data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        valid = torch.ones(batch_size, 1)
        fake = torch.zeros(batch_size, 1)
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(torch.rand([batch_size, 1], device=device))
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(fake_data), valid)
        g_loss.backward()
        optimizer_G.step()

        
        # Train generators
        optimizer_D.zero_grad()
        #d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data), dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #g_loss = adversarial_loss(F.log_softmax(fake_data, dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #noise_loss = adversarial_loss(F.log_softmax(fake_noise, dim=0), F.softmax(z1, dim=0), reduction='batchmean')
        
        

        fake_d_loss = adversarial_loss(discriminator(fake_data.detach()), fake)
        valid_d_loss = adversarial_loss(discriminator(price), valid)

        d_loss = 0.5 * (fake_d_loss + valid_d_loss)
        d_loss.backward()
        optimizer_G.step()

        g_loss_list.append(g_loss.item())
        d_loss_list.append(d_loss.item())

    optim_g_sched.step()
    optim_d_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {np.average(d_loss_list)}  Generator Loss: {np.average(g_loss_list)}")
    experiment.log_metric('Main Generator\Train', np.average(g_loss_list), step=epoch)
    experiment.log_metric('Discriminator\Train', np.average(d_loss_list), step=epoch)


In [18]:
# Test loop
batch_start = 0
for idx in range(int(len(test_data)/batch_size)-1):
    price = torch.tensor(test_data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
    price = price.to(device)
    batch_start = batch_start + batch_size
    # Generate random noise inputs
    z1 = cauchy_dist.sample([batch_size, 1])
    z1 = z1.to(device)
    #z2 = cauchy_dist.sample([1])
    main_gen.eval()
    noise_gen.eval()
    discriminator.eval()
    # Generate fake data
    fake_main = main_gen(price)
    fake_noise = noise_gen(torch.rand([batch_size, 1], device=device))
    fake_data = fake_main + fake_noise


    #d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data), dim=0), F.softmax(price, dim=0), reduction='batchmean')
    #g_loss = adversarial_loss(F.log_softmax(fake_data, dim=0), F.softmax(price, dim=0), reduction='batchmean')
    #noise_loss = adversarial_loss(F.log_softmax(fake_noise, dim=0), F.softmax(z1, dim=0), reduction='batchmean')
    d_loss = adversarial_loss(discriminator(fake_data), price)
    g_loss = adversarial_loss(price, fake_data)
    noise_loss = adversarial_loss(fake_noise, z1)
    total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
    optimizer_G.step()

optim_g_sched.step()
optim_d_sched.step()

print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
experiment.log_metric('Main Generator\Test', g_loss, step=epoch)
experiment.log_metric('Noise Generator\Test', noise_loss, step=epoch)
experiment.log_metric('Discriminator\Test', d_loss, step=epoch)
experiment.log_metric('Total\Test', total_loss, step=epoch)
# log_model(experiment, model=main_gen, model_name="Main Generator")
# log_model(experiment, model=noise_gen, model_name="Noise Generator")
# log_model(experiment, model=discriminator, model_name="Discriminator")


Epoch [100/100]  Discriminator Loss: 385699.9375  Generator Loss: 382206.8125


In [20]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/artaasd95/tail-price/29faec1725b4493eae8c3f8cb9469e86
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Discriminator\Test          : 385699.9375
COMET INFO:     Discriminator\Train [100]   : (8.331120317632502, 308944.53143544035)
COMET INFO:     Main Generator\Test         : 382206.8125
COMET INFO:     Main Generator\Train [100]  : (13899.041346526343, 328724.9913884943)
COMET INFO:     Noise Generator\Test        : 18.316530227661133
COMET INFO:     Noise Generator\Train [100] : (56.87691254064071, 4107405.4870138187)
COMET INFO:     Total\Test                  : 345734.53125
COMET INFO:     Total\Train [100]         

In [21]:
# Training loop
for epoch in tqdm(range(2)):
    batch_start = 0
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(data)/batch_size)-1):
        price = torch.tensor(data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(torch.rand([batch_size, 1], device=device))
        fake_data = fake_main + fake_noise
        #noise_loss = adversarial_loss(fake_noise, z1)
        # Train discriminator
        optimizer_G.zero_grad()
        #g_loss_data = adversarial_loss(fake_main, price)
        g_loss = adversarial_loss(discriminator(fake_data), valid)

        # g_loss_norm = F.normalize(torch.tensor([g_loss, g_loss_data]))
        # g_loss = torch.sum(g_loss_norm)
        g_loss.backward()
        optimizer_G.step()
        
        # Train generators
        optimizer_D.zero_grad()
        #d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data), dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #g_loss = adversarial_loss(F.log_softmax(fake_data, dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #noise_loss = adversarial_loss(F.log_softmax(fake_noise, dim=0), F.softmax(z1, dim=0), reduction='batchmean')
        
        

        fake_d_loss = adversarial_loss(discriminator(fake_data.detach()), fake)
        valid_d_loss = adversarial_loss(discriminator(price), valid)

        d_loss = 0.5 * (fake_d_loss + valid_d_loss)
        d_loss.backward()
        optimizer_D.step()

        g_loss_list.append(g_loss.item())
        d_loss_list.append(d_loss.item())

    optim_g_sched.step()
    optim_d_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {np.average(d_loss_list)}  Generator Loss: {np.average(g_loss_list)}")


  0%|          | 0/2 [08:33<?, ?it/s]


KeyboardInterrupt: 

In [15]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    batch_start = 0
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(data)/batch_size)-1):
        price = torch.tensor(data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(torch.rand([batch_size, 1], device=device))
        fake_data = fake_main + fake_noise
        
        noise_loss = adversarial_loss(fake_noise, z1)
        # Train discriminator
        optimizer_G.zero_grad()
        #g_loss_data = adversarial_loss(fake_main, price)
        g_loss = adversarial_loss(discriminator(fake_data), valid)

        # g_loss_norm = F.normalize(torch.tensor([g_loss, g_loss_data]))
        # g_loss = torch.sum(g_loss_norm)
        g_loss.backward()
        optimizer_G.step()
        
        # Train generators
        optimizer_D.zero_grad()
        #d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data), dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #g_loss = adversarial_loss(F.log_softmax(fake_data, dim=0), F.softmax(price, dim=0), reduction='batchmean')
        #noise_loss = adversarial_loss(F.log_softmax(fake_noise, dim=0), F.softmax(z1, dim=0), reduction='batchmean')
        
        

        fake_d_loss = adversarial_loss(discriminator(fake_data.detach()), fake)
        valid_d_loss = adversarial_loss(discriminator(price), valid)

        d_loss = 0.5 * (fake_d_loss + valid_d_loss)
        d_loss.backward()
        optimizer_D.step()

        g_loss_list.append(g_loss.item())
        d_loss_list.append(d_loss.item())

    optim_g_sched.step()
    optim_d_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {np.average(d_loss_list)}  Generator Loss: {np.average(g_loss_list)}")
    experiment.log_metric('Main Generator\Whole Train', np.average(g_loss_list), step=epoch)
    experiment.log_metric('Discriminator\Whole Train', np.average(d_loss_list), step=epoch)


  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\HP\.conda\envs\torchgpu\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  1%|          | 1/100 [01:06<1:49:55, 66.62s/it]

Epoch [1/100]  Discriminator Loss: 0.5318  Generator Loss: 1.0620


  2%|▏         | 2/100 [02:11<1:47:31, 65.83s/it]

Epoch [2/100]  Discriminator Loss: 0.5316  Generator Loss: 1.0625


  3%|▎         | 3/100 [03:17<1:46:04, 65.61s/it]

...

Epoch [98/100]  Discriminator Loss: 0.5319  Generator Loss: 1.0625


 99%|█████████▉| 99/100 [1:48:28<01:06, 66.22s/it]

Epoch [99/100]  Discriminator Loss: 0.5317  Generator Loss: 1.0618


100%|██████████| 100/100 [1:49:34<00:00, 65.75s/it]

Epoch [100/100]  Discriminator Loss: 0.5316  Generator Loss: 1.0625


In [16]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/artaasd95/tail-price/0e3b3779ac364200aa2f89e9bfba43c6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Discriminator\Whole Train [100]  : (0.5316601461068743, 0.5317050997232426)
COMET INFO:     Main Generator\Whole Train [100] : (1.0623610467580014, 1.0624684157399085)
COMET INFO:     loss [3460]                      : (0.5312371253967285, 1.0633294582366943)
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (46.91 KB)
COMET INFO:     installed packages       : 1
COMET

In [17]:
torch.save(main_gen, 'checkpoints/main_gen_xau.pth')
torch.save(noise_gen, 'checkpoints/noise_gen_xau.pth')
torch.save(discriminator, 'checkpoints/discriminator_xau.pth')